In [ ]:

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_capability())

True
(6, 0)


In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/IDA.zip" /content/

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/loadbm.py" /content/

In [ ]:
%%capture
!unzip IDA.zip

In [ ]:
import numpy as np
import os
import ntpath
import platform
import matplotlib.pyplot as plt
from loadbm import create_df, create_tf_dataset, prepare_for_training


datapath = 'IDA/Separate lists with numbering/Machine learning splits'
img_path = 'IDA/Images/'

split = 4

train_fname = 'train'+str(split)+'.txt'
test_fname = 'test'+str(split)+'.txt'
val_fname = 'val'+str(split)+'.txt'

part_dat = False

df_train = create_df(os.path.join(datapath, train_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_test = create_df(os.path.join(datapath, test_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_val = create_df(os.path.join(datapath, val_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

from sklearn.utils import shuffle

df_train = shuffle(df_train)
df_val = shuffle(df_val)

df_train.head()

,path,label
204358,IDA/Images/Micrasema_set12/1-Micrasema_set12.1...,24
170497,IDA/Images/Leptophlebia396/1-Leptophlebia396.1...,19
97622,IDA/Images/Heptagenia_sul205/0-Heptagenia_sul2...,11
14454,IDA/Images/Amphinemura_bor106/1-Amphinemura_bo...,3
6578,IDA/Images/Agapetus269/0-Agapetus269.1.2016-12...,1


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMSIZE = (224,224,3)
BATCH_SIZE = 32

train_ds = create_tf_dataset(df_train, imsize=IMSIZE, onehot=True)

val_ds = create_tf_dataset(df_val, imsize=IMSIZE, onehot=True)


train_ds = prepare_for_training(train_ds, 
                                shuffle_buffer_size=1000,
                                batch_size=BATCH_SIZE)

val_ds = prepare_for_training(val_ds, 
                              shuffle_buffer_size=1000,
                              batch_size=BATCH_SIZE)

for image, label in train_ds.take(5):
    print(image.shape)
    print(label.shape)


(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


def get_pretrained(imsize=(224, 224, 3), classes=39):
        base_model = InceptionV3(input_shape = imsize, 
                                 weights='imagenet', 
                                 include_top=False)
        
        base_model.trainable = True
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

model = get_pretrained()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics=['accuracy'])


87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.callbacks import CSVLogger
import datetime

tr_steps = len(df_train)//BATCH_SIZE
val_steps = len(df_val)//BATCH_SIZE

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 10,
          validation_steps = val_steps)

model.save('18-01-2020_cont_colab_split4.h5')

Epoch 1/10
10083/10083 [==============================] - 1665s 165ms/step - loss: 0.4487 - accuracy: 0.8571 - val_loss: 0.8530 - val_accuracy: 0.7851
Epoch 2/10
10083/10083 [==============================] - 1659s 165ms/step - loss: 0.1105 - accuracy: 0.9636 - val_loss: 0.8437 - val_accuracy: 0.8087
Epoch 3/10
10083/10083 [==============================] - 1657s 164ms/step - loss: 0.0637 - accuracy: 0.9795 - val_loss: 0.9379 - val_accuracy: 0.8025
Epoch 4/10
10083/10083 [==============================] - 1652s 164ms/step - loss: 0.0456 - accuracy: 0.9856 - val_loss: 0.5616 - val_accuracy: 0.8687
Epoch 5/10
10083/10083 [==============================] - 1650s 164ms/step - loss: 0.0355 - accuracy: 0.9886 - val_loss: 1.8063 - val_accuracy: 0.6717
Epoch 6/10
10083/10083 [==============================] - 1649s 164ms/step - loss: 0.0291 - accuracy: 0.9910 - val_loss: 0.4792 - val_accuracy: 0.8975
Epoch 7/10
10083/10083 [==============================] - 1649s 164ms/step - loss: 0.0252 - ac

In [ ]:
!cp '18-01-2020_cont_colab_split4.h5' "/content/drive/My Drive/koulu_honmia/kandi19/"

In [ ]:

#### Cont training #####
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.0001) #original 0.001

from tensorflow.keras.callbacks import LearningRateScheduler
def scheduler(epoch):
    return 0.0001 * tf.math.exp(0.1 * (-epoch))

lr_cb = LearningRateScheduler(scheduler)

model.compile(optimizer = adam, loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 5,
          validation_steps = val_steps,
          callbacks=[lr_cb])

Epoch 1/5
10083/10083 [==============================] - 1649s 164ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.4376 - val_accuracy: 0.9309 - lr: 1.0000e-04
Epoch 2/5
10083/10083 [==============================] - 1649s 164ms/step - loss: 5.4764e-04 - accuracy: 0.9998 - val_loss: 0.4679 - val_accuracy: 0.9316 - lr: 9.0484e-05
Epoch 3/5
10083/10083 [==============================] - 1648s 163ms/step - loss: 4.4364e-04 - accuracy: 0.9999 - val_loss: 0.4957 - val_accuracy: 0.9292 - lr: 8.1873e-05
Epoch 4/5
10083/10083 [==============================] - 1654s 164ms/step - loss: 4.0096e-04 - accuracy: 0.9999 - val_loss: 0.4900 - val_accuracy: 0.9325 - lr: 7.4082e-05
Epoch 5/5
10083/10083 [==============================] - 1657s 164ms/step - loss: 2.0642e-04 - accuracy: 0.9999 - val_loss: 0.5062 - val_accuracy: 0.9341 - lr: 6.7032e-05


In [ ]:
model.save('18-01-2020_cont_colab_split4.h5')

In [ ]:
!cp '18-01-2020_cont_colab_split4.h5' "/content/drive/My Drive/koulu_honmia/kandi19/"

Evaluation:

In [ ]:
test_ds = create_tf_dataset(df_test, imsize=IMSIZE, onehot=True)
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/combine_insects.py" /content/

In [ ]:
preds = model.predict(test_ds, verbose=True)
yhat = np.argmax(preds,axis=1)+1
y_test = df_test['label']

acc = np.sum(yhat==y_test)/len(y_test)
print('Image accuracy: {:.4f}'.format(acc))

np.save('18-01-2020_cont_colab_split4.npy', preds)

2876/2876 [==============================] - 173s 60ms/step
Image accuracy: 0.9273


In [ ]:
!cp '18-01-2020_cont_colab_split4.npy' "/content/drive/My Drive/koulu_honmia/kandi19/" 

In [ ]:
#%% Insect combine
from combine_insects import add_insect_class, add_yhat

df_test_preds = add_insect_class(df_test)

# adding predictions to dataframe for insect-wise prediction
df_test_preds = add_yhat(df_test_preds,yhat)

dfg = df_test_preds.groupby(['label','insect'],as_index=False)['pred'].agg(lambda x:x.value_counts().index[0])

acc_g = np.sum(dfg['pred']==dfg['label'])/len(dfg)
print('Aggregate accuracy: {:.4f}'.format(acc_g))

Aggregate accuracy: 0.9545
